# Tutorial 1b: Generate hard-links using PASTE

In [10]:
import math
import time
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import style
import paste as pst

In [8]:
import os

data_dir = "../../spatial_benchmarking/benchmarking_data/DLPFC12"
section_ids = ['151507', '151508']

def load_DLPFC(root_dir='../benchmarking_data/DLPFC12', section_id='151507'):
    # 151507, ..., 151676 12 in total
    ad = sc.read_visium(path=os.path.join(root_dir, section_id), count_file=section_id+'_filtered_feature_bc_matrix.h5')
    ad.var_names_make_unique()

    gt_dir = os.path.join(root_dir, section_id, 'gt')
    gt_df = pd.read_csv(os.path.join(gt_dir, 'tissue_positions_list_GTs.txt'), sep=',', header=None, index_col=0)
    ad.obs['original_clusters'] = gt_df.loc[:, 6]
    keep_bcs = ad.obs.dropna().index
    ad = ad[keep_bcs].copy()
    ad.obs['original_clusters'] = ad.obs['original_clusters'].astype(int).astype(str)
    # print(ad.obs)
    return ad

Batch_list = []
for section_id in section_ids:
    ad_ = load_DLPFC(root_dir=data_dir, section_id=section_id)
    ad_.var_names_make_unique(join="++")

    # make spot name unique
    ad_.obs_names = [x+'_'+section_id for x in ad_.obs_names]
    
    
    sc.pp.filter_genes(ad_, min_counts = 15)
    sc.pp.filter_cells(ad_, min_counts = 100)
    ad_.obsm['spatial'] = ad_.obsm['spatial'].to_numpy()

    Batch_list.append(ad_)

slice1, slice2 = Batch_list

In [ ]:
import ot
start = time.time()

pi12 = pst.pairwise_align(slice1, slice2, use_gpu=True, backend = ot.backend.TorchBackend())

In [ ]:
pis = [pi12]
slices = [slice1, slice2]

new_slices = pst.stack_slices_pairwise(slices, pis)

In [ ]:
import pandas as pd

for i, L in enumerate(new_slices):
    spatial_data = L.obsm['spatial']

    output_path = os.path.join('./', f"coordinates_{section_ids[i]}.csv")
    pd.DataFrame(spatial_data).to_csv(output_path, index=False)
    print(f"Saved spatial data for slice {i} to {output_path}")